In [1]:
from xml.dom import minidom
! pip install gensim
import gensim.downloader as api
model = api.load('glove-twitter-100')

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [ ]:
mydoc = minidom.parse('./pan22-author-profiling-training-2022-03-29/en/11790334350e58e6c92fdeb1d10bc161.xml')

print(mydoc)


In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('./pan22-author-profiling-training-2022-03-29/en/11790334350e58e6c92fdeb1d10bc161.xml')
root = tree.getroot()

print(root[0][10].text)

In [ ]:
#number of tweets in a file
N = len(root[0])


In [ ]:
for n in range(0,N):
    print(root[0][n].text)

In [ ]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

data_tok = []
for i in range(0,N):  
    j=str(root[0][i].text)
    data_tok.append([k.lower() for k in tokenizer.tokenize(j)])


In [ ]:
import numpy as np
def get_phrase_embedding(phrase):

    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    phrase = phrase.lower()
    phrase = tokenizer.tokenize(phrase)
    
    phrase_vectors = []
    
    for i in phrase:
        if i in model.key_to_index.keys():
            phrase_vectors.append(model.get_vector(i))
    
    phrase_vectors = np.array(phrase_vectors)
    
    if len(phrase_vectors) == 0:
        return vector
    
    phrase_vectors = np.mean(phrase_vectors, axis=0)
    
    return phrase_vectors


In [ ]:
def get_person_vector(phrase):
    data = []
    N = len(phrase[0])
    for n in range(0,N):
        vector = get_phrase_embedding(str(phrase[0][n].text))
        data.append(vector)
    data1 = np.array(data).reshape(20000,)#.reshape(20000,)
   # print(data1.shape)
    return data1
    
vector = get_person_vector(root)
#print(len(vector[0]))

In [11]:
import os
path = "./pan22-author-profiling-training-2022-03-29/en" 
test_path = "./pan22-author-profiling-training-2022-03-29/en"
files= os.listdir(path) 
test_files = os.listdir(test_path) 
data = []
GT = "./pan22-author-profiling-training-2022-03-29/en/truth.txt"
true_values = {}
f=open(GT)
for line in f:
    linev = line.strip().split(":::")
    true_values[linev[0]] = linev[1]
f.close()

for file in files: 
    if str(os.path.basename(file)) == 'truth.txt':
        continue
    if not os.path.isdir(file):
        tree = ET.parse(path+"/"+file)
        root = tree.getroot()
        vector = get_person_vector(root)
        
        if true_values[str(os.path.basename(file)).strip('.xml')]=='I':
            data.append([vector,0])
        else:
            data.append([vector,1])
    
        

In [9]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from scipy import signal

import matplotlib.pyplot as plt
import scipy.io as sio
! pip install neurokit2
import neurokit2 as nk
! pip install seaborn
import seaborn as sns

import pandas as pd
import numpy as np
import time

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [21]:
from sklearn.model_selection import train_test_split
df = pd.DataFrame(data) 
train, test = train_test_split(df, test_size=60)
X_train=list(train[0].values)
X_test = list(test[0].values)
y_train=train[1].values
y_test = test[1].values
print(train.shape,test.shape)

(360, 2) (60, 2)


In [22]:
def run_clf(clf):
    score = []
    runtime = []
    clf.fit(X_train, y_train)
    start = time.time()
    score.append(clf.score(X_test, y_test))
    runtime.append(time.time() - start)

    return score, runtime

In [23]:
results = []
names = ["Nearest Neighbors", "Linear SVM",
         "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest",
         "Neural Net", "AdaBoost",
         "Naive Bayes"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5,
                           n_estimators=10,
                           max_features=1),
    MLPClassifier(alpha=1,
                  max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB()]

for name, classifier in zip(names, classifiers):
    clf = make_pipeline(MinMaxScaler(), classifier)
    score, runtime = run_clf(clf)
    results.append([name,
                    np.mean(score),
                    np.mean(runtime)])

In [ ]:
results_df = pd.DataFrame(results, columns=['name',
                                            'mean_score',
                                            'mean_runtime'])
results_df.to_csv('results.csv')

In [ ]:
results_df

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.DataFrame(data) 
train, test = train_test_split(df, test_size=60)
model_svm = SVC(kernel='linear', probability=True)
#train1=train[0].values.reshape(360,-1)
model_svm.fit(list(train[0].values),train[1].values)

pre1 = model_svm.predict(list(test[0].values))
print(pre1)
print(sum(pre1==test[1].values)/60)


[0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0
 0 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 1 1 0]
0.8833333333333333


In [14]:
import torch
import torch.nn as nn
import torch.utils.data as Data

class LSTM(nn.Module):
    def __init__(self, input_size=20000, hidden_layer_size=100, output_size=1):
       
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_x):
        input_x = input_x.view(len(input_x), 1, -1)
        hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),  # shape: (n_layers, batch, hidden_size)
                       torch.zeros(1, 1, self.hidden_layer_size))
        lstm_out, (h_n, h_c) = self.lstm(input_x, hidden_cell)
        linear_out = self.linear(lstm_out.view(len(input_x), -1))  # =self.linear(lstm_out[:, -1, :])
        predictions = self.sigmoid(linear_out)
        return predictions


In [30]:
from sklearn import preprocessing
import numpy as np
a = np.ones((420,200,100))
b = df[0].values
for i in range(420):
    a[i]=b[i].reshape(200,100)
print(a.shape)
    
x, y = torch.from_numpy(a.astype(float)).to(torch.float32), torch.from_numpy(np.array(df[1].values)).to(torch.float32)

train_loader = Data.DataLoader(
        dataset=Data.TensorDataset(x, y), 
        batch_size=1,  
        shuffle=True,  
        num_workers=2, 
    )
lstm = LSTM()  
loss_function = nn.BCELoss()  # loss
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001)  
epochs = 10
    
lstm.train()
for i in range(epochs):
    for seq, labels in train_loader:
        optimizer.zero_grad()
        y_pred = lstm(seq).squeeze()  
        labels = labels.squeeze()
        single_loss = loss_function(y_pred, labels)
           
        single_loss.backward()
        optimizer.step()
       
lstm.eval()

for seq, labels in train_loader:  
    y_pred = lstm(seq).squeeze()  
    labels = labels.squeeze()
    single_loss = loss_function(y_pred, labels)
    acc = sum(y_pred==labels)/len(y_pred)
#    print("EVAL Step:", i, " accuracy: ", acc)



(420, 200, 100)


ValueError: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.

In [24]:
a = np.ones((60,200,100))
b = test[0].values
for i in range(60):
    a[i]=b[i].reshape(200,100)
x, y = torch.from_numpy(a.astype(float)).to(torch.float32), torch.from_numpy(np.array(train[1].values)).to(torch.float32)
y_test_pred = lstm(x).squeeze()  
result = []
for i in y_test_pred:
    if i >0.5:
        result.append(1)
    else:
        result.append(0)
print(result)

acc = sum(result==test[1].values)/60
print(acc)

[0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0]
0.85
